In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler, MinMaxScaler
import time
import datetime

# 创建一个Spark会话
spark = SparkSession.builder.appName("MySparkSession").getOrCreate()

# 加载Parquet文件
df = spark.read.parquet('../time_series/TS6/TS6-2022.parquet')
df = df.select("__index_level_0__", *df.columns[:-1])
df.show()

+-----------------+----+----+-----+---+----+------------+-------+----------+-----+-----------+------------+-------------------+------------------+
|__index_level_0__|Name|year|month|day|hour|PULocationID|weekday|is_holiday|count|        lat|         lon|           TempTime|            countN|
+-----------------+----+----+-----+---+----+------------+-------+----------+-----+-----------+------------+-------------------+------------------+
|         20013480|lyft|2022|    1|  1|   0|           3|      5|      true|   16|40.86429404|-73.84650986|2022-01-01 00:00:00|16.666666666666668|
|         20013481|lyft|2022|    1|  1|   1|           3|      5|      true|   21|40.86429404|-73.84650986|2022-01-01 01:00:00|              14.0|
|         20013482|lyft|2022|    1|  1|   2|           3|      5|      true|    6|40.86429404|-73.84650986|2022-01-01 02:00:00| 6.666666666666667|
|         20013483|lyft|2022|    1|  1|   3|           3|      5|      true|   11|40.86429404|-73.84650986|2022-01-01 

#### 將布林值轉成0跟1 

In [2]:
from pyspark.sql.functions import col
df = df.withColumn("is_holiday", col("is_holiday").cast("int"))
df.show()

+-----------------+----+----+-----+---+----+------------+-------+----------+-----+-----------+------------+-------------------+------------------+
|__index_level_0__|Name|year|month|day|hour|PULocationID|weekday|is_holiday|count|        lat|         lon|           TempTime|            countN|
+-----------------+----+----+-----+---+----+------------+-------+----------+-----+-----------+------------+-------------------+------------------+
|         20013480|lyft|2022|    1|  1|   0|           3|      5|         1|   16|40.86429404|-73.84650986|2022-01-01 00:00:00|16.666666666666668|
|         20013481|lyft|2022|    1|  1|   1|           3|      5|         1|   21|40.86429404|-73.84650986|2022-01-01 01:00:00|              14.0|
|         20013482|lyft|2022|    1|  1|   2|           3|      5|         1|    6|40.86429404|-73.84650986|2022-01-01 02:00:00| 6.666666666666667|
|         20013483|lyft|2022|    1|  1|   3|           3|      5|         1|   11|40.86429404|-73.84650986|2022-01-01 

In [3]:
from pyspark.sql.functions import when
from pyspark.sql.types import IntegerType
mapping = {
    'lyft': 2,
    'uber': 3,
    'yellow': 1
}

# 使用when和otherwise函数进行替换
for key, value in mapping.items():
    df = df.withColumn("Name", when(df["Name"] == key, value).otherwise(df["Name"]))
    
df = df.withColumn("Name", df["Name"].cast(IntegerType()))    
df.show()

+-----------------+----+----+-----+---+----+------------+-------+----------+-----+-----------+------------+-------------------+------------------+
|__index_level_0__|Name|year|month|day|hour|PULocationID|weekday|is_holiday|count|        lat|         lon|           TempTime|            countN|
+-----------------+----+----+-----+---+----+------------+-------+----------+-----+-----------+------------+-------------------+------------------+
|         20013480|   2|2022|    1|  1|   0|           3|      5|         1|   16|40.86429404|-73.84650986|2022-01-01 00:00:00|16.666666666666668|
|         20013481|   2|2022|    1|  1|   1|           3|      5|         1|   21|40.86429404|-73.84650986|2022-01-01 01:00:00|              14.0|
|         20013482|   2|2022|    1|  1|   2|           3|      5|         1|    6|40.86429404|-73.84650986|2022-01-01 02:00:00| 6.666666666666667|
|         20013483|   2|2022|    1|  1|   3|           3|      5|         1|   11|40.86429404|-73.84650986|2022-01-01 

In [4]:
# 删除不需要的列
df = df.drop("TempTime")
df.show()

+-----------------+----+----+-----+---+----+------------+-------+----------+-----+-----------+------------+------------------+
|__index_level_0__|Name|year|month|day|hour|PULocationID|weekday|is_holiday|count|        lat|         lon|            countN|
+-----------------+----+----+-----+---+----+------------+-------+----------+-----+-----------+------------+------------------+
|         20013480|   2|2022|    1|  1|   0|           3|      5|         1|   16|40.86429404|-73.84650986|16.666666666666668|
|         20013481|   2|2022|    1|  1|   1|           3|      5|         1|   21|40.86429404|-73.84650986|              14.0|
|         20013482|   2|2022|    1|  1|   2|           3|      5|         1|    6|40.86429404|-73.84650986| 6.666666666666667|
|         20013483|   2|2022|    1|  1|   3|           3|      5|         1|   11|40.86429404|-73.84650986| 6.333333333333333|
|         20013484|   2|2022|    1|  1|   4|           3|      5|         1|   10|40.86429404|-73.84650986| 6.3

In [5]:
from pyspark.sql import SparkSession


# 假设您已经加载了 DataFrame df

# 获取 DataFrame 的行数
row_count = df.count()

# 获取 DataFrame 的列数
column_count = len(df.columns)

# 打印行数和列数
print("行数:", row_count)
print("列数:", column_count)


行数: 6859080
列数: 13


In [6]:
df1 = spark.read.parquet('../time_series/TS6/TS6-2023.parquet')
df1 = df1.select("__index_level_0__", *df1.columns[:-1])
df1.show()

df1 = df1.withColumn("is_holiday", col("is_holiday").cast("int"))
df1.show()

from pyspark.sql.functions import when
mapping = {
    'lyft': 2,
    'uber': 3,
    'yellow': 1
}

# 使用when和otherwise函数进行替换
for key, value in mapping.items():
    df1 = df1.withColumn("Name", when(df1["Name"] == key, value).otherwise(df1["Name"]))
df1 = df1.withColumn("Name", df1["Name"].cast(IntegerType()))        
df1.show()

df1 = df1.drop("TempTime")
df1.show()

# 假设您已经加载了 DataFrame df

# 获取 DataFrame 的行数
row_count = df1.count()

# 获取 DataFrame 的列数
column_count = len(df1.columns)

# 打印行数和列数
print("行数:", row_count)
print("列数:", column_count)


+-----------------+----+----+-----+---+----+------------+-------+----------+-----+-----------+------------+-------------------+------------------+
|__index_level_0__|Name|year|month|day|hour|PULocationID|weekday|is_holiday|count|        lat|         lon|           TempTime|            countN|
+-----------------+----+----+-----+---+----+------------+-------+----------+-----+-----------+------------+-------------------+------------------+
|         26872560|lyft|2023|    1|  1|   0|           3|      6|      true|   12|40.86429404|-73.84650986|2023-01-01 00:00:00|              15.0|
|         26872561|lyft|2023|    1|  1|   1|           3|      6|      true|   28|40.86429404|-73.84650986|2023-01-01 01:00:00|15.333333333333334|
|         26872562|lyft|2023|    1|  1|   2|           3|      6|      true|   19|40.86429404|-73.84650986|2023-01-01 02:00:00|              14.0|
|         26872563|lyft|2023|    1|  1|   3|           3|      6|      true|   26|40.86429404|-73.84650986|2023-01-01 

行数: 3401352
列数: 13


In [7]:
from pyspark.sql import SparkSession

# 创建一个Spark会话
spark = SparkSession.builder.appName("MySparkSession").getOrCreate()

# 假设您已经加载了 DataFrame df

# 筛选训练集数据
train_data = df.filter(df["month"] < 11)
train_data.show()

# 筛选测试集数据
test_data = df.filter(df["month"] > 10)
test_data.show()

val_data = df1
val_data.show()

+-----------------+----+----+-----+---+----+------------+-------+----------+-----+-----------+------------+------------------+
|__index_level_0__|Name|year|month|day|hour|PULocationID|weekday|is_holiday|count|        lat|         lon|            countN|
+-----------------+----+----+-----+---+----+------------+-------+----------+-----+-----------+------------+------------------+
|         20013480|   2|2022|    1|  1|   0|           3|      5|         1|   16|40.86429404|-73.84650986|16.666666666666668|
|         20013481|   2|2022|    1|  1|   1|           3|      5|         1|   21|40.86429404|-73.84650986|              14.0|
|         20013482|   2|2022|    1|  1|   2|           3|      5|         1|    6|40.86429404|-73.84650986| 6.666666666666667|
|         20013483|   2|2022|    1|  1|   3|           3|      5|         1|   11|40.86429404|-73.84650986| 6.333333333333333|
|         20013484|   2|2022|    1|  1|   4|           3|      5|         1|   10|40.86429404|-73.84650986| 6.3

In [8]:
# 提取特征列和标签列
feature_columns = df.columns
feature_columns.remove("count")
feature_columns.remove("countN")

X_train = train_data.select(feature_columns)
y_train = train_data.select("countN")
X_test = test_data.select(feature_columns)
y_test = test_data.select("count")
X_val = val_data.select(feature_columns)
y_val = val_data.select("count")

In [9]:
X_train.dtypes

[('__index_level_0__', 'bigint'),
 ('Name', 'int'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint'),
 ('hour', 'bigint'),
 ('PULocationID', 'bigint'),
 ('weekday', 'int'),
 ('is_holiday', 'int'),
 ('lat', 'double'),
 ('lon', 'double')]

In [10]:
X_test.dtypes

[('__index_level_0__', 'bigint'),
 ('Name', 'int'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint'),
 ('hour', 'bigint'),
 ('PULocationID', 'bigint'),
 ('weekday', 'int'),
 ('is_holiday', 'int'),
 ('lat', 'double'),
 ('lon', 'double')]

In [11]:
X_val.dtypes

[('__index_level_0__', 'bigint'),
 ('Name', 'int'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint'),
 ('hour', 'bigint'),
 ('PULocationID', 'bigint'),
 ('weekday', 'int'),
 ('is_holiday', 'int'),
 ('lat', 'double'),
 ('lon', 'double')]

In [12]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
import time
import datetime

# 输出开始时间
print("start時間:", datetime.datetime.now())
print('------------------')

# 创建VectorAssembler来合并特征列为一个Vector列
assembler = VectorAssembler(inputCols=X_train.columns, outputCol="features")
assembler_test = VectorAssembler(inputCols=X_test.columns, outputCol="features_test")
X_train = assembler.transform(X_train)
X_test = assembler_test.transform(X_test)
X_val = assembler_test.transform(X_val)


# 创建 MinMaxScaler
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scaler_test = MinMaxScaler(inputCol="features_test", outputCol="scaled_features_test")


# 估计缩放器的参数
scaler_model = scaler.fit(X_train)
scaler_model_test = scaler_test.fit(X_test)

# 使用缩放器将数据进行转换
X_train_scaled = scaler_model.transform(X_train)
X_test_scaled = scaler_model_test.transform(X_test)
X_val_scaled = scaler_model_test.transform(X_val)

# 添加标签列到DataFrame
y_train_scaled = y_train.withColumnRenamed("countN", "label")
y_test_scaled = y_test.withColumnRenamed("count", "label")
y_val_scaled = y_val.withColumnRenamed("count", "label")


y_train_scaled.show()


start時間: 2023-09-05 10:32:43.270686
------------------
+------------------+
|             label|
+------------------+
|16.666666666666668|
|              14.0|
| 6.666666666666667|
| 6.333333333333333|
| 6.333333333333333|
| 5.333333333333333|
|               6.0|
| 4.666666666666667|
| 7.666666666666667|
|               9.0|
| 6.333333333333333|
| 5.666666666666667|
|10.333333333333334|
|10.333333333333334|
|11.666666666666666|
|14.666666666666666|
|12.666666666666666|
|              15.0|
|14.666666666666666|
|              10.0|
+------------------+
only showing top 20 rows



In [13]:
X_train_scaled.select('scaled_features').show()

+--------------------+
|     scaled_features|
+--------------------+
|[0.0,0.5,0.5,0.0,...|
|[1.54393061575812...|
|[3.08786123151625...|
|[4.63179184727438...|
|[6.17572246303251...|
|[7.71965307879063...|
|[9.26358369454876...|
|[1.08075143103068...|
|[1.23514449260650...|
|[1.38953755418231...|
|[1.54393061575812...|
|[1.69832367733394...|
|[1.85271673890975...|
|[2.00710980048556...|
|[2.16150286206137...|
|[2.31589592363719...|
|[2.47028898521300...|
|[2.62468204678881...|
|[2.77907510836463...|
|[2.93346816994044...|
+--------------------+
only showing top 20 rows



In [14]:
train_data = X_train_scaled.select('scaled_features').rdd.flatMap(lambda x: x).collect()


# 创建随机森林回归模型
rforest = RandomForestRegressor(numTrees=1000, seed=0, minInstancesPerNode=20)

# 创建一个Pipeline来执行特征向量化和模型训练
# pipeline = Pipeline(stages=[rforest])

# 计时开始
start_time = time.time()

# 训练模型
model = rforest.fit(train_data)

# 停止计时
end_time = time.time()
execution_time = end_time - start_time
print("\n程序执行花费的时间：", round(execution_time, 2), "秒")
print("done時間:", datetime.datetime.now())


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 48.0 failed 1 times, most recent failure: Lost task 3.0 in stage 48.0 (TID 161) (P215-2203-NB01 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:192)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:166)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:92)
	at org.apach

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler

# 使用VectorAssembler创建特征向量
assembler = VectorAssembler(inputCols=X_train.columns, outputCol="features")
X_train = assembler.transform(X_train)
X_test = assembler.transform(X_test)
X_val = assembler.transform(X_val)

# 创建 MinMaxScaler
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")

# 估计缩放器的参数
scaler_model = scaler.fit(X_train)

# 使用缩放器将数据进行转换
X_train_scaled = scaler_model.transform(X_train)
X_test_scaled = scaler_model.transform(X_test)
X_val_scaled = scaler_model.transform(X_val)
